## Bike Index Seattle - Data Prep

### Data cleaning for crash data

##### Objective: Recreate the study by Allen-Munley et al. (2004) for Seattle using WSDOT crash data.


#### Part 1.

The crash data .csv file from WSDOT contains all crash reports for the entire state of Washington. I will clean up the .csv file to keep only collisions involving bicycles, and will focus only within the Seattle city limits.   

I will also keep just `ROAD SURFACE CONDITIONS`, `LIGHTING CONDITIONS`, `WEATHER`, `HIT & RUN` categorical variables, and will further dummy code these as binary variables.  

Collision severity will be mapped to a 1-4 severity index similar to what was used in Allen-Munley et al.'s study.  

I will be keeping `BLOCK NUMBER` to allow for merging with other road data for topography, road widths, etc.

In [1]:
import numpy as np
import pandas as pd
import os
import folium
from folium import plugins

#### Step 1 - Load data

- Read .csv file
- Filter to just relevant subset (bike collisions, Seattle only)
- Drop unwanted columns

In [2]:
crash_data = pd.read_csv("../../UW/DATA511/Final Project/20201103Yamauchi_All_roads_Statewide_SRFF.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,5,6,8,9,11,12,13,42,48,49,50,51,67,68,84,85,96,99,101,102,110,111,116,126,128,129,134,143,144,145,146,147,151,152,155,156,163,164,167,169,170,171,172,173,174,175,176,177,179,180,181,182,183,184,185,186,188,189,190,191,192,193,194,195,197,198,199,200,203) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
crash_data.head()

,JURISDICTION,COUNTY,CITY,REPORT NUMBER,INDEXED PRIMARY TRAFFICWAY,PRIMARY TRAFFICWAY,BLOCK NUMBER,MILEPOST,A/B,INTERSECTING TRAFFICWAY,...,TZ Heavy Vehicle Crash Indicator,TZ Heavy Vehicle Crash Count,TZ Vehicle Train Crash Indicator,TZ Catostrophic Event Indicator,TZ Fatal Crash Indicator,TZ Fatality Count,TZ Suspected Serious Injury Crash Indicator,TZ Suspected Serious Injury Count,TZ Pedestrian Involved Indicator,TZ Pedacyclist Involved Indicator
0,City Street,Adams,Othello,E713622,ALLEY E OF S 12TH AV,ALLEY E OF S 12TH AVE,400,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,City Street,Adams,Othello,E999637,ALLEYWAY NORTH OF MA,ALLEYWAY NORTH OF MAIN,900,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,City Street,Adams,Othello,E962138,ASH ST,ASH ST,1200,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,City Street,Adams,Othello,EA21607,CAPSTONE AVE,CAPSTONE AVE,1000,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,City Street,Adams,Othello,E916903,CAPSTONE AVE,CAPSTONE AVE,0,NaN,NaN,GEMSTONE ST,...,0,0,0,0,0,0,0,0,0,0


In [4]:
crash_data.describe()

,MILEPOST,DIST FROM REF POINT,YEAR,TOTAL CRASHES,FATAL CRASHES,SERIOUS INJURY CRASHES,EVIDENT INJURY CRASHES,POSSIBLE INJURY CRASHES,PDO - NO INJURY CRASHES,TOTAL FATALITIES,...,TZ Heavy Vehicle Crash Indicator,TZ Heavy Vehicle Crash Count,TZ Vehicle Train Crash Indicator,TZ Catostrophic Event Indicator,TZ Fatal Crash Indicator,TZ Fatality Count,TZ Suspected Serious Injury Crash Indicator,TZ Suspected Serious Injury Count,TZ Pedestrian Involved Indicator,TZ Pedacyclist Involved Indicator
count,256574.000000,95914.000000,448711.000000,448711.0,448711.000000,448711.000000,448711.000000,448711.000000,448711.000000,448711.000000,...,448711.000000,448711.000000,448711.000000,448711.000000,448711.000000,448711.000000,448711.000000,448711.000000,448711.000000,448711.000000
mean,48.790287,151.375624,2018.021339,1.0,0.004593,0.017392,0.069925,0.210811,0.697295,0.004896,...,0.056306,0.059731,0.000368,0.000432,0.004569,0.004872,0.017417,0.020271,0.019436,0.010795
std,79.105791,151.958862,1.186822,0.0,0.067617,0.130727,0.255020,0.407885,0.459429,0.074766,...,0.230511,0.252169,0.019173,0.020789,0.067437,0.074603,0.130818,0.158605,0.138051,0.103339
min,-0.570000,0.000000,2016.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.250000,52.000000,2017.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8.800000,107.000000,2018.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,52.730000,203.000000,2019.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,765.000000,4558.000000,2020.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,...,1.000000,9.000000,1.000000,1.000000,1.000000,4.000000,1.000000,6.000000,1.000000,1.000000


In [5]:
crash_data.columns

Index(['JURISDICTION', 'COUNTY', 'CITY', 'REPORT NUMBER',
       'INDEXED PRIMARY TRAFFICWAY', 'PRIMARY TRAFFICWAY', 'BLOCK NUMBER',
       'MILEPOST', 'A/B ', 'INTERSECTING TRAFFICWAY',
       ...
       'TZ Heavy Vehicle Crash Indicator', 'TZ Heavy Vehicle Crash Count',
       'TZ Vehicle Train Crash Indicator', 'TZ Catostrophic Event Indicator',
       'TZ Fatal Crash Indicator', 'TZ Fatality Count',
       'TZ Suspected Serious Injury Crash Indicator',
       'TZ Suspected Serious Injury Count', 'TZ Pedestrian Involved Indicator',
       'TZ Pedacyclist Involved Indicator'],
      dtype='object', length=255)

In [6]:
df = crash_data.loc[crash_data['CITY'] == 'Seattle']

In [7]:
df.describe()

,MILEPOST,DIST FROM REF POINT,YEAR,TOTAL CRASHES,FATAL CRASHES,SERIOUS INJURY CRASHES,EVIDENT INJURY CRASHES,POSSIBLE INJURY CRASHES,PDO - NO INJURY CRASHES,TOTAL FATALITIES,...,TZ Heavy Vehicle Crash Indicator,TZ Heavy Vehicle Crash Count,TZ Vehicle Train Crash Indicator,TZ Catostrophic Event Indicator,TZ Fatal Crash Indicator,TZ Fatality Count,TZ Suspected Serious Injury Crash Indicator,TZ Suspected Serious Injury Count,TZ Pedestrian Involved Indicator,TZ Pedacyclist Involved Indicator
count,16286.000000,14005.000000,45793.000000,45793.0,45793.000000,45793.000000,45793.000000,45793.000000,45793.000000,45793.000000,...,45793.000000,45793.000000,45793.000000,45793.000000,45793.000000,45793.000000,45793.000000,45793.000000,45793.000000,45793.000000
mean,86.896353,116.860425,2017.949490,1.0,0.002162,0.016334,0.080864,0.244732,0.655908,0.002206,...,0.067303,0.071321,0.001223,0.000131,0.002162,0.002206,0.016334,0.017514,0.042408,0.029131
std,79.161309,106.045908,1.159778,0.0,0.046446,0.126759,0.272629,0.429933,0.475076,0.047834,...,0.250549,0.273888,0.034949,0.011446,0.046446,0.047834,0.126759,0.139252,0.201521,0.168176
min,0.000000,0.000000,2016.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.220000,50.000000,2017.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,39.945000,98.000000,2018.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,166.080000,150.000000,2019.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,174.570000,4000.000000,2020.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,...,1.000000,5.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000


In [8]:
keep_cols = [
    'REPORT NUMBER','BLOCK NUMBER','DATE','24 HR TIME',
    'TOTAL FATALITIES','TOTAL SERIOUS INJURIES','TOTAL EVIDENT INJURIES','TOTAL POSSIBLE INJURIES','PDO - NO INJURY CRASHES',
    'TOTAL VEHICLES','TOTAL PEDESTRIANS INVOLVED','TOTAL BICYCLISTS INVOLVED',
    'WEATHER','ROAD SURFACE CONDITIONS','LIGHTING CONDITIONS','HIT & RUN',
    'WA STATE PLANE SOUTH - X','WA STATE PLANE SOUTH - Y'
]

In [9]:
df_bike = df.loc[df['TOTAL BICYCLISTS INVOLVED'] > 0]
df_bike = df_bike[df_bike.columns.intersection(keep_cols)].reset_index(drop = True)

#### Step 2: Check map visualization of the data

- Convert state plane coordinates to Lat/Lon coordinates
- Visualize on map coordinates

In [10]:
from pyproj import Transformer

In [11]:
x,y = df_bike['WA STATE PLANE SOUTH - X'], df_bike['WA STATE PLANE SOUTH - Y']
transformer = Transformer.from_crs('epsg:2286','epsg:4326')
lat,long = transformer.transform(x, y)
df_bike['LONGITUDE'] = long
df_bike['LATITUDE'] = lat

In [12]:
m = folium.Map(location = [47.6062, -122.3321], zoom_start = 13)
accidents = plugins.MarkerCluster().add_to(m)

for i in range(len(df_bike)):
    lat = df_bike['LATITUDE']
    long = df_bike['LONGITUDE']
    folium.CircleMarker(
        location = [lat[i], long[i]],
        color = 'red',
        fill = True,
    ).add_to(accidents)

m

#### Step 3: Convert data 

- Convert date/time columns to single datetime variable
- Convert categorical data into binary dummy variables:
    - Road conditions: 1 if `Dry` else 0
    - Lighting conditions: 1 if `Daylight` else 0
    - Weather: 1 if `Clear` or `Clear or Partly Cloudy` else 0
    - Hit & Run: 1 if `Yes` else 0

In [13]:
df_bike["DATETIME"] = pd.to_datetime(df_bike["DATE"] + " " + df_bike["24 HR TIME"])

In [17]:
df_bike['ROAD SURFACE CONDITIONS'].unique()

array(['Wet', 'Dry', 'Unknown', 'Ice', 'Standing Water', 'Snow/Slush'],
      dtype=object)

In [14]:
df_bike['is_dry'] = [1 if surface == 'Dry' else 0 for surface in df_bike['ROAD SURFACE CONDITIONS']]

In [16]:
df_bike['LIGHTING CONDITIONS'].unique()

array(['Daylight', 'Dark-Street Lights On', 'Dusk', 'Dawn', 'Other',
       'Dark-No Street Lights', 'Dark - Unknown Lightin', 'Unknown',
       'Dark-Street Lights Off'], dtype=object)

In [18]:
df_bike['is_light'] = [1 if lighting == 'Daylight' else 0 for lighting in df_bike['LIGHTING CONDITIONS']]

In [21]:
df_bike['WEATHER'].unique()

array(['Raining', 'Clear or Partly Cloudy', 'Unknown', 'Overcast',
       'Clear', 'Other', 'Blowing Sand or Dirt or Snow',
       'Fog or Smog or Smoke', nan, 'Snowing'], dtype=object)

In [22]:
df_bike['is_clear'] = [1 if weather in ['Clear or Partly Cloudy', 'Clear'] else 0 for weather in df_bike['WEATHER']]

In [59]:
df_bike['HIT & RUN'].unique()

array(['No', 'Yes'], dtype=object)

In [60]:
df_bike['is_hit_run'] = [1 if hitrun == 'Yes' else 0 for hitrun in df_bike['HIT & RUN']]

#### Step 4: Map dummified injury severity to single severity index

The Allen-Munley et al. study uses a 1-3 index (there were no fatalities in their sample).  

![Severity Index Distribution](severity_index.png)  

The injuries will be mapped using the following index: 
- `PDO - NO INJURY CRASHES` : 1
- `POSSIBLE INJURIES` : 2
- `EVIDENT INJURIES` : 3
- `SERIOUS INJURIES` : 3
- `FATALITIES` : 4

In [46]:
severity_cols = [
    'PDO - NO INJURY CRASHES',
    'TOTAL FATALITIES',
    'TOTAL SERIOUS INJURIES',
    'TOTAL EVIDENT INJURIES',
    'TOTAL POSSIBLE INJURIES'
]


severity_dict = {
    'PDO - NO INJURY CRASHES' : 1,
    'TOTAL POSSIBLE INJURIES' : 2,
    'TOTAL EVIDENT INJURIES' : 3,
    'TOTAL SERIOUS INJURIES' : 3,
    'TOTAL FATALITIES' : 4
}

In [55]:
severity_df = df_bike[severity_cols].copy()

sev_sers = pd.Series(severity_df.columns[np.where(np.array(severity_df)!=0)[1]])

severity_df.isna().sum()

PDO - NO INJURY CRASHES    0
TOTAL FATALITIES           0
TOTAL SERIOUS INJURIES     0
TOTAL EVIDENT INJURIES     0
TOTAL POSSIBLE INJURIES    0
dtype: int64

In [56]:
sev_cat = sev_sers.map(severity_dict)

df_bike['severity'] = sev_cat

In [62]:
df_bike.columns

Index(['REPORT NUMBER', 'BLOCK NUMBER', 'DATE', '24 HR TIME',
       'PDO - NO INJURY CRASHES', 'TOTAL FATALITIES', 'TOTAL SERIOUS INJURIES',
       'TOTAL EVIDENT INJURIES', 'TOTAL POSSIBLE INJURIES', 'TOTAL VEHICLES',
       'TOTAL PEDESTRIANS INVOLVED', 'TOTAL BICYCLISTS INVOLVED', 'WEATHER',
       'ROAD SURFACE CONDITIONS', 'LIGHTING CONDITIONS', 'HIT & RUN',
       'WA STATE PLANE SOUTH - X', 'WA STATE PLANE SOUTH - Y', 'LONGITUDE',
       'LATITUDE', 'DATETIME', 'is_dry', 'is_light', 'is_clear', 'severity',
       'is_hit_run'],
      dtype='object')

In [63]:
keep_cols_2 = [
    'REPORT NUMBER','BLOCK NUMBER','DATETIME','LATITUDE','LONGITUDE',
    'severity','is_dry','is_light','is_clear','is_hit_run',
    'TOTAL VEHICLES','TOTAL PEDESTRIANS INVOLVED','TOTAL BICYCLISTS INVOLVED'
]

In [64]:
df_bike = df_bike[keep_cols_2].copy()

In [66]:
df_bike.head()

,REPORT NUMBER,BLOCK NUMBER,DATETIME,LATITUDE,LONGITUDE,severity,is_dry,is_light,is_clear,is_hit_run,TOTAL VEHICLES,TOTAL PEDESTRIANS INVOLVED,TOTAL BICYCLISTS INVOLVED
0,3773772,0,2019-04-19 15:52:00,47.606205,-122.319417,3,0,1,0,0,1,0,1
1,3773784,700,2017-06-27 06:40:00,47.626561,-122.320235,3,1,1,1,0,1,0,1
2,E779051,2100,2018-03-10 23:00:00,47.638665,-122.320075,3,0,0,0,1,1,0,1
3,3773767,1900,2017-07-02 15:13:00,47.636483,-122.320093,3,1,1,1,0,1,0,1
4,EA10570,600,2020-01-31 13:45:00,47.624503,-122.319881,3,0,1,0,0,1,0,1


#### Step 5: Write to .csv

Write current cleaned dataframe to .csv file.  
I will next gather roadway data to merge to this dataframe to replicate the road-specific variables used in the study.

In [67]:
df_bike.to_csv('data/bike_crash.csv')